# Consumo do modelo de Advanced Analitics

`keywords.: serving, deploy`

Este notebook tem as instruções que simulam o consumo de um modelo de machine learning em ambiente do AWS SageMaker.


## Setup

Seção com a preparação das variáveis do processo de predição.

`Ref. Tempo esperado de execução: 2,15s`


In [2]:
%%time

# Célula de configurações

import os
import boto3
import re
import json
from sagemaker import get_execution_role, session

region = boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

# Você pode usar um bucket, mas certifique-se de que a permissão que escolheu para este notebook
# contempla s3:PutObject . Este é o bucket no qual os dados são capturados
bucket = session.Session(boto3.Session()).default_bucket()
print("Bucket a ser usado: {}".format(bucket))
prefix = "sagemaker/DEMO-ModelMonitor"

data_capture_prefix = "{}/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)
reports_prefix = "{}/reports".format(prefix)
s3_report_path = "s3://{}/{}".format(bucket, reports_prefix)
code_prefix = "{}/code".format(prefix)
s3_code_preprocessor_uri = "s3://{}/{}/{}".format(bucket, code_prefix, "preprocessor.py")
s3_code_postprocessor_uri = "s3://{}/{}/{}".format(bucket, code_prefix, "postprocessor.py")

print("Path de captura dos dados: {}".format(s3_capture_upload_path))
print("Path dos relatórios: {}".format(s3_report_path))
print("Path para código de pré-proc: {}".format(s3_code_preprocessor_uri))
print("Path para código de pós-proc: {}".format(s3_code_postprocessor_uri))
print("\n")

RoleArn: arn:aws:iam::325011675573:role/service-role/AmazonSageMaker-ExecutionRole-20201008T200444
Bucket a ser usado: sagemaker-us-east-2-325011675573
Path de captura dos dados: s3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/datacapture
Path dos relatórios: s3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/reports
Path para código de pré-proc: s3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/code/preprocessor.py
Path para código de pós-proc: s3://sagemaker-us-east-2-325011675573/sagemaker/DEMO-ModelMonitor/code/postprocessor.py


CPU times: user 711 ms, sys: 140 ms, total: 851 ms
Wall time: 1.31 s


In [3]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
import time

## Chamada do modelo

Agora vamos enviar dados a este endpoint para obter inferências em tempo real. Como ativamos a captura de dados nas etapas anteriores, a carga útil de solicitação e resposta, junto com alguns metadados adicionais, é salva no local do Amazon Simple Storage Service (Amazon S3) que você especificou no DataCaptureConfig.

Esta etapa chama o processo de scoring para 180 registros de exemplo. Os dados são capturados com base na porcentagem de amostragem especificada e a captura continua até que a opção de captura de dados seja desligada.

In [4]:
endpoint_name = "DEMO-xgb-churn-pred-model-monitor-2021-09-16-16-37-14"

In [12]:
predictor = Predictor(endpoint_name=endpoint_name, serializer=CSVSerializer())

# Obtém um subconjunto de dados de teste para um teste rápido
!head -180 test_data/test-dataset-input-cols.csv > test_data/test_sample.csv
print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

i = 0
with open("test_data/test_sample.csv", "r") as f:
    for row in f:
        i = i + 1
        response = predictor.predict(data=payload)
        
        payload = row.rstrip("\n")
        if i == 1:
            print("Primeiro pedido [{}]:\n".format(i))
            print(payload)
            print("\n Resposta: {} \n".format(response))
        
print("Último pedido [{}]:\n".format(i))
print(payload)
print("\nResposta: {} \n".format(response))



Sending test traffic to the endpoint DEMO-xgb-churn-pred-model-monitor-2021-09-16-16-37-14. 
Please wait...
Primeiro pedido [1]:

186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,1.1,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,0.51,0.52,0.53,1.2,1.3,0.54,1.4,0.55

 Resposta: b'0.04491584002971649' 

Último pedido [180]:

113,32,180.4,89,129.4,124,166.9,124,8.4,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1

Resposta: b'0.03876146301627159' 



## Ver os dados capturados

Agora listamos os arquivos de captura de dados armazenados no Amazon S3.

Você deve esperar para ver diferentes arquivos de diferentes períodos de tempo organizados com base na hora em que a chamada ocorreu. O formato do caminho do Amazon S3 é:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [13]:
s3_client = boto3.Session().client("s3")
current_endpoint_capture_prefix = "{}/{}".format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
sagemaker/DEMO-ModelMonitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2021-09-16-16-37-14/AllTraffic/2021/09/16/18/27-57-976-96fceca0-900a-44c2-bba0-73127e65c70f.jsonl
 sagemaker/DEMO-ModelMonitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2021-09-16-16-37-14/AllTraffic/2021/09/16/18/29-30-321-2260c27f-64cb-4fa9-9936-66ebc0043683.jsonl


Em seguida, veremos o conteúdo de um único arquivo de captura. Aqui podemos ver todos os dados capturados em um arquivo formatado em linha JSON específico do Amazon SageMaker.

Dê uma olhada rápida nas primeiras linhas do arquivo capturado (em texto em em json).

In [15]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get("Body").read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"113,32,180.4,89,129.4,124,166.9,124,8.4,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.04491584002971649","encoding":"CSV"}},"eventMetadata":{"eventId":"6d32fec1-a7df-4198-8430-9186f3294258","inferenceTime":"2021-09-16T18:29:30Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,1.1,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,0.51,0.52,0.53,1.2,1.3,0.54,1.4,0.55","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/

Finally, the contents of a single line is present below in a formatted JSON file so that you can observe a little better.

In [16]:
import json

print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "113,32,180.4,89,129.4,124,166.9,124,8.4,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.04491584002971649",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "6d32fec1-a7df-4198-8430-9186f3294258",
    "inferenceTime": "2021-09-16T18:29:30Z"
  },
  "eventVersion": "0"
}


Como você pode ver, cada solicitação de inferência é capturada em uma linha no arquivo json. A linha contém a entrada e a saída mescladas. No exemplo, fornecemos o ContentType como `text / csv`, que é refletido no valor `observedContentType`. Além disso, expomos a codificação que usou para codificar as cargas úteis de entrada e saída no formato de captura com o valor `encoding`.

Para recapitular, você observou como pode ativar a captura das cargas úteis de entrada ou saída para um ponto de extremidade com um novo parâmetro. Você também observou a aparência do formato capturado no Amazon S3. Em seguida, continue a explorar como o Amazon SageMaker ajuda a monitorar os dados coletados no Amazon S3.

---